# Decision Tree Assignment - Part 1: Theoretical Understanding

## 📚 Assignment Overview

This notebook covers the theoretical foundations of Decision Trees through comprehensive questions and practical implementations. We'll explore core concepts, mathematical foundations, and develop intuition for tree-based algorithms.

## 🎯 Learning Objectives

By completing this assignment, you will:
- Understand the fundamental concepts of Decision Trees
- Master the mathematical foundations (Entropy, Information Gain, Gini Impurity)
- Learn when and how to apply Decision Trees effectively
- Develop intuition for tree-based algorithms

---

## Part 1: Theoretical Questions

**Q1.** Explain the concept of a Decision Tree. What kind of problems is it best suited for?

**Q2.** Define the following terms with examples:
- Root Node
- Leaf Node  
- Internal Node
- Branch

**Q3.** What is Entropy? How is it used in a Decision Tree? Provide a mathematical example.

**Q4.** What is Information Gain? How does it help in building a Decision Tree? Show a small example using a feature with two values.

**Q5.** Compare Gini Impurity and Entropy as criteria in Decision Trees. When would you prefer one over the other?

## 1. Import Required Libraries

Let's start by importing the necessary libraries for our theoretical exploration and visualizations.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.datasets import make_classification
import math
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")
print("✅ NumPy for numerical computations")
print("✅ Pandas for data manipulation")
print("✅ Matplotlib & Seaborn for visualizations")
print("✅ Scikit-learn for Decision Tree implementation")
print("✅ Math for mathematical calculations")

## 2. Q1: Decision Tree Concepts and Applications

### Question 1: Explain the concept of a Decision Tree. What kind of problems is it best suited for?

### Answer:

A **Decision Tree** is a supervised machine learning algorithm that uses a tree-like structure to make predictions. It works by recursively splitting the data based on feature values, creating a hierarchical set of if-else conditions that lead to predictions.

#### Key Characteristics:
1. **Tree Structure**: Organized as a tree with nodes and branches
2. **Recursive Partitioning**: Splits data based on feature values
3. **Interpretability**: Easy to understand and visualize
4. **Non-parametric**: Makes no assumptions about data distribution

#### Problems Best Suited For:

**Classification Problems:**
- Email spam detection
- Medical diagnosis
- Customer segmentation
- Fraud detection

**Regression Problems:**
- House price prediction
- Stock price forecasting
- Sales prediction

#### Advantages:
- ✅ Highly interpretable
- ✅ Handles both numerical and categorical data
- ✅ No need for feature scaling
- ✅ Can capture non-linear relationships
- ✅ Handles missing values well

#### When to Use Decision Trees:
- When interpretability is crucial
- Mixed data types (numerical + categorical)
- Non-linear relationships exist
- Feature interactions are important

In [ ]:
# Practical Example: Decision Tree for Classification
# Let's create a simple example to demonstrate Decision Tree concepts

# Create a sample dataset
np.random.seed(42)
data = {
    'Weather': ['Sunny', 'Overcast', 'Rainy', 'Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Sunny', 'Rainy'],
    'Temperature': ['Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Strong'],
    'Play_Tennis': ['No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes']
}

df = pd.DataFrame(data)
print("Sample Dataset - Tennis Playing Decision:")
print(df)

# Visualize the decision scenario
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Count plot for target variable
target_counts = df['Play_Tennis'].value_counts()
ax1.pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Distribution of Play Tennis Decision')

# Feature importance visualization
feature_counts = {}
for col in ['Weather', 'Temperature', 'Humidity', 'Wind']:
    unique_vals = df[col].nunique()
    feature_counts[col] = unique_vals

ax2.bar(feature_counts.keys(), feature_counts.values(), color=['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon'])
ax2.set_title('Number of Unique Values per Feature')
ax2.set_ylabel('Count')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\nDataset Shape: {df.shape}")
print(f"Features: {list(df.columns[:-1])}")
print(f"Target: {df.columns[-1]}")
print(f"Target Classes: {df['Play_Tennis'].unique()}")

## 3. Q2: Tree Components and Terminology

### Question 2: Define the following terms with examples:
- Root Node
- Leaf Node
- Internal Node
- Branch

### Answer:

#### 1. **Root Node**
- **Definition**: The topmost node in a decision tree where the tree starts
- **Characteristics**: 
  - Contains the entire dataset
  - Represents the first decision/split
  - Has no parent nodes
  - Has one or more child nodes

**Example**: In a medical diagnosis tree, the root node might be "Patient has fever?" which divides all patients into two groups.

#### 2. **Leaf Node (Terminal Node)**
- **Definition**: End nodes that contain the final prediction/classification
- **Characteristics**:
  - No child nodes
  - Contains the predicted class or value
  - Represents the final decision

**Example**: In our tennis example, leaf nodes would be "Play Tennis: Yes" or "Play Tennis: No"

#### 3. **Internal Node (Decision Node)**
- **Definition**: Intermediate nodes that represent a test/condition on a feature
- **Characteristics**:
  - Has both parent and child nodes
  - Contains a decision rule
  - Splits the data based on feature values

**Example**: "Humidity = High?" or "Weather = Sunny?" are internal nodes that further divide the data.

#### 4. **Branch (Edge)**
- **Definition**: Connections between nodes representing the outcome of a test
- **Characteristics**:
  - Links parent node to child node
  - Represents a specific condition or value
  - Shows the path of decision making

**Example**: The branch labeled "Yes" connects "Humidity = High?" to the next decision node.

In [ ]:
# Visualizing Tree Components with Python
# Create a simple decision tree to demonstrate different components

# Prepare the data for sklearn (encode categorical variables)
from sklearn.preprocessing import LabelEncoder

# Create a copy of our dataset
df_encoded = df.copy()

# Encode categorical variables
label_encoders = {}
for column in ['Weather', 'Temperature', 'Humidity', 'Wind', 'Play_Tennis']:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Separate features and target
X = df_encoded[['Weather', 'Temperature', 'Humidity', 'Wind']]
y = df_encoded['Play_Tennis']

# Create and fit the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X, y)

# Visualize the tree structure
plt.figure(figsize=(20, 10))
plot_tree(dt, 
          feature_names=['Weather', 'Temperature', 'Humidity', 'Wind'],
          class_names=['No', 'Yes'], 
          filled=True, 
          rounded=True, 
          fontsize=12)

plt.title("Decision Tree Structure - Identifying Different Node Types", fontsize=16, fontweight='bold')
plt.show()

# Print tree information
print("🌳 Tree Structure Analysis:")
print(f"├── Root Node: Feature '{X.columns[dt.tree_.feature[0]]}' (Node 0)")
print(f"├── Tree Depth: {dt.get_depth()}")
print(f"├── Number of Nodes: {dt.tree_.node_count}")
print(f"├── Number of Leaves: {dt.get_n_leaves()}")

# Identify different types of nodes
def analyze_tree_structure(tree, node=0, depth=0):
    if tree.children_left[node] == tree.children_right[node]:  # Leaf node
        print("  " * depth + f"🍃 Leaf Node {node}: Prediction = {tree.value[node].argmax()}")
    else:  # Internal node
        if node == 0:
            print("  " * depth + f"🌱 Root Node {node}: Split on feature {tree.feature[node]}")
        else:
            print("  " * depth + f"🔸 Internal Node {node}: Split on feature {tree.feature[node]}")
        
        # Recursively analyze children
        analyze_tree_structure(tree, tree.children_left[node], depth + 1)
        analyze_tree_structure(tree, tree.children_right[node], depth + 1)

print("\n📊 Node Type Analysis:")
analyze_tree_structure(dt.tree_)

## 4. Q3: Entropy Calculation and Examples

### Question 3: What is Entropy? How is it used in a Decision Tree? Provide a mathematical example.

### Answer:

#### **What is Entropy?**

**Entropy** is a measure of impurity or randomness in a dataset. In the context of decision trees, it quantifies how mixed the classes are in a given node.

#### **Mathematical Formula:**

For a dataset with classes C₁, C₂, ..., Cₖ:

**Entropy(S) = -∑(i=1 to k) pᵢ × log₂(pᵢ)**

Where:
- S = dataset
- pᵢ = proportion of samples that belong to class Cᵢ
- k = number of classes

#### **Key Properties:**
- **Range**: 0 ≤ Entropy ≤ log₂(k)
- **Entropy = 0**: Pure node (all samples belong to same class)
- **Entropy = log₂(k)**: Maximum impurity (equal distribution of all classes)

#### **How Entropy is Used in Decision Trees:**

1. **Node Splitting**: Choose the feature that results in the lowest weighted average entropy
2. **Stopping Criteria**: Stop splitting when entropy reaches minimum threshold
3. **Information Gain**: Calculate the reduction in entropy after splitting

#### **Mathematical Example:**

Let's consider a node with 10 samples:
- 7 samples of Class A (positive)
- 3 samples of Class B (negative)

**Step-by-step calculation:**

In [ ]:
# Entropy Calculation Implementation

def calculate_entropy(labels):
    """
    Calculate entropy for a given set of labels
    """
    if len(labels) == 0:
        return 0
    
    # Count occurrences of each class
    class_counts = Counter(labels)
    total_samples = len(labels)
    
    # Calculate entropy
    entropy = 0
    for count in class_counts.values():
        probability = count / total_samples
        if probability > 0:  # Avoid log(0)
            entropy -= probability * math.log2(probability)
    
    return entropy

# Mathematical Example from the theory above
print("📊 Mathematical Example - Entropy Calculation")
print("=" * 50)

# Example dataset: 7 Class A, 3 Class B
example_labels = ['A'] * 7 + ['B'] * 3
print(f"Dataset: {example_labels}")
print(f"Class A: 7 samples, Class B: 3 samples")
print(f"Total: {len(example_labels)} samples")

# Manual calculation
p_a = 7/10  # Probability of Class A
p_b = 3/10  # Probability of Class B

print(f"\nStep-by-step calculation:")
print(f"p(A) = 7/10 = {p_a}")
print(f"p(B) = 3/10 = {p_b}")
print(f"")
print(f"Entropy = -p(A) × log₂(p(A)) - p(B) × log₂(p(B))")
print(f"Entropy = -{p_a} × log₂({p_a}) - {p_b} × log₂({p_b})")
print(f"Entropy = -{p_a} × {math.log2(p_a):.4f} - {p_b} × {math.log2(p_b):.4f}")
print(f"Entropy = {-p_a * math.log2(p_a):.4f} + {-p_b * math.log2(p_b):.4f}")

manual_entropy = -p_a * math.log2(p_a) - p_b * math.log2(p_b)
function_entropy = calculate_entropy(example_labels)

print(f"Entropy = {manual_entropy:.4f}")
print(f"\nVerification using our function: {function_entropy:.4f}")
print(f"✅ Results match: {abs(manual_entropy - function_entropy) < 1e-10}")

# Demonstrate different entropy scenarios
print(f"\n🎯 Different Entropy Scenarios:")
print("=" * 50)

scenarios = [
    (['A'] * 10, "Pure node (all Class A)"),
    (['A'] * 5 + ['B'] * 5, "Maximum impurity (50-50 split)"),
    (['A'] * 8 + ['B'] * 2, "Low impurity (80-20 split)"),
    (['A'] * 6 + ['B'] * 3 + ['C'] * 1, "Three classes"),
]

entropies = []
scenario_names = []

for labels, description in scenarios:
    entropy = calculate_entropy(labels)
    entropies.append(entropy)
    scenario_names.append(description)
    print(f"{description:.<35} Entropy = {entropy:.4f}")

# Visualize entropy for different scenarios
plt.figure(figsize=(12, 8))

# Plot entropy values
plt.subplot(2, 2, 1)
bars = plt.bar(range(len(entropies)), entropies, color=['green', 'red', 'orange', 'purple'])
plt.xlabel('Scenario')
plt.ylabel('Entropy')
plt.title('Entropy Values for Different Scenarios')
plt.xticks(range(len(entropies)), [f'S{i+1}' for i in range(len(entropies))])

# Add value labels on bars
for i, (bar, entropy) in enumerate(zip(bars, entropies)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{entropy:.3f}', ha='center', va='bottom')

# Plot entropy curve for binary classification
plt.subplot(2, 2, 2)
p_values = np.linspace(0.01, 0.99, 100)
entropy_values = [-p * math.log2(p) - (1-p) * math.log2(1-p) for p in p_values]

plt.plot(p_values, entropy_values, 'b-', linewidth=2)
plt.xlabel('Probability of Class 1')
plt.ylabel('Entropy')
plt.title('Entropy Curve for Binary Classification')
plt.grid(True, alpha=0.3)
plt.axhline(y=1, color='r', linestyle='--', alpha=0.7, label='Maximum Entropy')
plt.legend()

# Tennis dataset entropy
plt.subplot(2, 2, 3)
tennis_entropy = calculate_entropy(df['Play_Tennis'].tolist())
play_counts = df['Play_Tennis'].value_counts()

plt.pie(play_counts.values, labels=play_counts.index, autopct='%1.1f%%', startangle=90)
plt.title(f'Tennis Dataset\nEntropy = {tennis_entropy:.4f}')

# Information visualization
plt.subplot(2, 2, 4)
info_text = f"""
Entropy Interpretation:

• Entropy = 0: Pure node
• Entropy = 1: Maximum impurity 
  (for binary classification)
• Lower entropy = Better separation
• Used to calculate Information Gain

Tennis Dataset:
• Total samples: {len(df)}
• Yes: {play_counts['Yes']} samples
• No: {play_counts['No']} samples
• Entropy: {tennis_entropy:.4f}
"""

plt.text(0.1, 0.9, info_text, transform=plt.gca().transAxes, 
         verticalalignment='top', fontsize=10, 
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
plt.axis('off')

plt.tight_layout()
plt.show()

## 5. Q4: Information Gain Implementation

### Question 4: What is Information Gain? How does it help in building a Decision Tree? Show a small example using a feature with two values.

### Answer:

#### **What is Information Gain?**

**Information Gain** is a measure used to determine which feature provides the most information about the target variable. It calculates the reduction in entropy after splitting the dataset on a particular feature.

#### **Mathematical Formula:**

**Information Gain(S, A) = Entropy(S) - Σ(|Sᵥ|/|S|) × Entropy(Sᵥ)**

Where:
- S = original dataset
- A = feature being evaluated
- Sᵥ = subset of S where feature A has value v
- |S| = number of samples in S
- |Sᵥ| = number of samples in Sᵥ

#### **How Information Gain Helps in Building Decision Trees:**

1. **Feature Selection**: Choose the feature with the highest information gain for splitting
2. **Tree Construction**: Recursively apply this process to build the tree
3. **Optimal Splits**: Ensures each split provides maximum information about the target
4. **Greedy Approach**: Makes locally optimal choices at each node

#### **Step-by-Step Example:**

Let's use a simple example with a binary feature "Weather" that has two values: "Sunny" and "Rainy".

In [ ]:
# Information Gain Implementation

def calculate_information_gain(data, target_col, feature_col):
    """
    Calculate information gain for a specific feature
    """
    # Calculate entropy of the original dataset
    original_entropy = calculate_entropy(data[target_col].tolist())
    
    # Get unique values of the feature
    feature_values = data[feature_col].unique()
    
    # Calculate weighted entropy after splitting
    weighted_entropy = 0
    total_samples = len(data)
    
    for value in feature_values:
        subset = data[data[feature_col] == value]
        subset_size = len(subset)
        subset_entropy = calculate_entropy(subset[target_col].tolist())
        
        # Add weighted entropy
        weighted_entropy += (subset_size / total_samples) * subset_entropy
    
    # Calculate information gain
    information_gain = original_entropy - weighted_entropy
    
    return information_gain, original_entropy, weighted_entropy

# Create a simple example dataset for Information Gain demonstration
print("🎯 Information Gain Example - Binary Feature")
print("=" * 55)

# Create example data with Weather feature (Sunny/Rainy) and Play Tennis target
example_data = pd.DataFrame({
    'Weather': ['Sunny', 'Sunny', 'Rainy', 'Rainy', 'Sunny', 'Rainy', 'Sunny', 'Rainy'],
    'Play_Tennis': ['No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes']
})

print("Example Dataset:")
print(example_data)
print()

# Analyze the dataset
weather_counts = example_data.groupby(['Weather', 'Play_Tennis']).size().unstack(fill_value=0)
print("Cross-tabulation of Weather vs Play Tennis:")
print(weather_counts)
print()

# Calculate Information Gain step by step
print("📊 Step-by-Step Information Gain Calculation:")
print("-" * 50)

# Step 1: Calculate original entropy
original_labels = example_data['Play_Tennis'].tolist()
original_entropy = calculate_entropy(original_labels)
print(f"Step 1 - Original Entropy:")
print(f"Dataset: {original_labels}")
total_yes = original_labels.count('Yes')
total_no = original_labels.count('No')
total_samples = len(original_labels)
print(f"Yes: {total_yes}, No: {total_no}, Total: {total_samples}")
print(f"Entropy(S) = {original_entropy:.4f}")
print()

# Step 2: Calculate entropy for each subset
print(f"Step 2 - Calculate Entropy for Each Subset:")

# Sunny subset
sunny_data = example_data[example_data['Weather'] == 'Sunny']
sunny_labels = sunny_data['Play_Tennis'].tolist()
sunny_entropy = calculate_entropy(sunny_labels)
sunny_yes = sunny_labels.count('Yes')
sunny_no = sunny_labels.count('No')
sunny_total = len(sunny_labels)

print(f"Sunny subset: {sunny_labels}")
print(f"  Yes: {sunny_yes}, No: {sunny_no}, Total: {sunny_total}")
print(f"  Entropy(Sunny) = {sunny_entropy:.4f}")

# Rainy subset  
rainy_data = example_data[example_data['Weather'] == 'Rainy']
rainy_labels = rainy_data['Play_Tennis'].tolist()
rainy_entropy = calculate_entropy(rainy_labels)
rainy_yes = rainy_labels.count('Yes')
rainy_no = rainy_labels.count('No')
rainy_total = len(rainy_labels)

print(f"Rainy subset: {rainy_labels}")
print(f"  Yes: {rainy_yes}, No: {rainy_no}, Total: {rainy_total}")
print(f"  Entropy(Rainy) = {rainy_entropy:.4f}")
print()

# Step 3: Calculate weighted entropy
print(f"Step 3 - Calculate Weighted Entropy:")
sunny_weight = sunny_total / total_samples
rainy_weight = rainy_total / total_samples

print(f"Weight(Sunny) = {sunny_total}/{total_samples} = {sunny_weight:.3f}")
print(f"Weight(Rainy) = {rainy_total}/{total_samples} = {rainy_weight:.3f}")

weighted_entropy = sunny_weight * sunny_entropy + rainy_weight * rainy_entropy
print(f"Weighted Entropy = {sunny_weight:.3f} × {sunny_entropy:.4f} + {rainy_weight:.3f} × {rainy_entropy:.4f}")
print(f"Weighted Entropy = {weighted_entropy:.4f}")
print()

# Step 4: Calculate Information Gain
print(f"Step 4 - Calculate Information Gain:")
info_gain = original_entropy - weighted_entropy
print(f"Information Gain = Entropy(S) - Weighted Entropy")
print(f"Information Gain = {original_entropy:.4f} - {weighted_entropy:.4f}")
print(f"Information Gain = {info_gain:.4f}")
print()

# Verify with our function
ig, orig_ent, weight_ent = calculate_information_gain(example_data, 'Play_Tennis', 'Weather')
print(f"✅ Verification using function: {ig:.4f}")
print(f"   Original Entropy: {orig_ent:.4f}")
print(f"   Weighted Entropy: {weight_ent:.4f}")

# Visualize the Information Gain concept
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Original dataset distribution
original_counts = example_data['Play_Tennis'].value_counts()
ax1.pie(original_counts.values, labels=original_counts.index, autopct='%1.1f%%', startangle=90)
ax1.set_title(f'Original Dataset\nEntropy = {original_entropy:.4f}')

# After splitting by Weather
weather_groups = example_data.groupby('Weather')['Play_Tennis'].apply(list)

sunny_counts = pd.Series(weather_groups['Sunny']).value_counts()
ax2.pie(sunny_counts.values, labels=sunny_counts.index, autopct='%1.1f%%', startangle=90)
ax2.set_title(f'Sunny Days\nEntropy = {sunny_entropy:.4f}')

rainy_counts = pd.Series(weather_groups['Rainy']).value_counts()
ax3.pie(rainy_counts.values, labels=rainy_counts.index, autopct='%1.1f%%', startangle=90)
ax3.set_title(f'Rainy Days\nEntropy = {rainy_entropy:.4f}')

# Information Gain visualization
ax4.bar(['Original\nEntropy', 'Weighted\nEntropy', 'Information\nGain'], 
        [original_entropy, weighted_entropy, info_gain],
        color=['red', 'orange', 'green'])
ax4.set_ylabel('Value')
ax4.set_title('Information Gain Calculation')

# Add value labels
for i, v in enumerate([original_entropy, weighted_entropy, info_gain]):
    ax4.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate Information Gain for all features in the Tennis dataset
print("\n🏆 Information Gain for All Features in Tennis Dataset")
print("=" * 60)

features = ['Weather', 'Temperature', 'Humidity', 'Wind']
information_gains = []

print("Feature Rankings by Information Gain:")
print("-" * 40)

for feature in features:
    ig, orig_ent, weight_ent = calculate_information_gain(df, 'Play_Tennis', feature)
    information_gains.append(ig)
    print(f"{feature:12} | Info Gain: {ig:.4f}")

# Find the best feature
best_feature_idx = np.argmax(information_gains)
best_feature = features[best_feature_idx]
best_gain = information_gains[best_feature_idx]

print(f"\n🎯 Best feature for root node: {best_feature}")
print(f"   Information Gain: {best_gain:.4f}")

# Visualize Information Gain comparison
plt.figure(figsize=(12, 6))

# Bar plot of Information Gains
plt.subplot(1, 2, 1)
bars = plt.bar(features, information_gains, color=['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon'])
plt.title('Information Gain Comparison')
plt.ylabel('Information Gain')
plt.xticks(rotation=45)

# Highlight the best feature
bars[best_feature_idx].set_color('gold')
bars[best_feature_idx].set_edgecolor('red')
bars[best_feature_idx].set_linewidth(2)

# Add value labels
for i, (feature, gain) in enumerate(zip(features, information_gains)):
    plt.text(i, gain + 0.005, f'{gain:.4f}', ha='center', va='bottom')

# Feature selection decision tree
plt.subplot(1, 2, 2)
feature_data = pd.DataFrame({
    'Feature': features,
    'Information_Gain': information_gains
}).sort_values('Information_Gain', ascending=True)

plt.barh(feature_data['Feature'], feature_data['Information_Gain'], 
         color=['lightcoral' if x != best_gain else 'gold' for x in feature_data['Information_Gain']])
plt.title('Feature Selection Ranking')
plt.xlabel('Information Gain')

# Add value labels
for i, (feature, gain) in enumerate(zip(feature_data['Feature'], feature_data['Information_Gain'])):
    plt.text(gain + 0.002, i, f'{gain:.4f}', ha='left', va='center')

plt.tight_layout()
plt.show()

print(f"\n💡 Interpretation:")
print(f"   The '{best_feature}' feature provides the most information")
print(f"   about the target variable and should be used at the root node.")
print(f"   This feature reduces entropy by {best_gain:.4f} bits.")

## 6. Q5: Gini Impurity vs Entropy Comparison

### Question 5: Compare Gini Impurity and Entropy as criteria in Decision Trees. When would you prefer one over the other?

### Answer:

#### **Gini Impurity**

**Definition**: Gini Impurity measures the probability of incorrectly classifying a randomly chosen element if it were randomly labeled according to the distribution of labels in the node.

**Mathematical Formula:**
**Gini(S) = 1 - Σ(i=1 to k) pᵢ²**

Where:
- S = dataset
- pᵢ = proportion of samples that belong to class Cᵢ
- k = number of classes

#### **Entropy**

**Definition**: Entropy measures the average amount of information needed to identify the class of a sample.

**Mathematical Formula:**
**Entropy(S) = -Σ(i=1 to k) pᵢ × log₂(pᵢ)**

#### **Key Differences:**

| Aspect | Gini Impurity | Entropy |
|--------|---------------|---------|
| **Range** | 0 to 0.5 (binary) | 0 to 1 (binary) |
| **Computation** | Faster (no logarithm) | Slower (logarithm calculation) |
| **Sensitivity** | Less sensitive to changes | More sensitive to changes |
| **Pure Node** | Gini = 0 | Entropy = 0 |
| **Maximum Impurity** | Gini = 0.5 | Entropy = 1 |

#### **When to Use Each:**

**Use Gini Impurity when:**
- ✅ Computational efficiency is important
- ✅ Working with large datasets
- ✅ Implementation simplicity is preferred
- ✅ Slight differences in purity don't matter much

**Use Entropy when:**
- ✅ More balanced trees are desired
- ✅ Theoretical foundation is important
- ✅ Maximum information gain is crucial
- ✅ Better handling of probability distributions

In [ ]:
# Gini Impurity vs Entropy Implementation and Comparison

def calculate_gini_impurity(labels):
    """
    Calculate Gini impurity for a given set of labels
    """
    if len(labels) == 0:
        return 0
    
    # Count occurrences of each class
    class_counts = Counter(labels)
    total_samples = len(labels)
    
    # Calculate Gini impurity
    gini = 1.0
    for count in class_counts.values():
        probability = count / total_samples
        gini -= probability ** 2
    
    return gini

def calculate_gini_gain(data, target_col, feature_col):
    """
    Calculate Gini gain for a specific feature
    """
    # Calculate Gini impurity of the original dataset
    original_gini = calculate_gini_impurity(data[target_col].tolist())
    
    # Get unique values of the feature
    feature_values = data[feature_col].unique()
    
    # Calculate weighted Gini impurity after splitting
    weighted_gini = 0
    total_samples = len(data)
    
    for value in feature_values:
        subset = data[data[feature_col] == value]
        subset_size = len(subset)
        subset_gini = calculate_gini_impurity(subset[target_col].tolist())
        
        # Add weighted Gini impurity
        weighted_gini += (subset_size / total_samples) * subset_gini
    
    # Calculate Gini gain
    gini_gain = original_gini - weighted_gini
    
    return gini_gain, original_gini, weighted_gini

print("⚖️  Gini Impurity vs Entropy Comparison")
print("=" * 50)

# Compare both measures on the same example dataset
example_labels = ['A'] * 7 + ['B'] * 3

print("Example Dataset: 7 Class A, 3 Class B")
print("-" * 40)

# Calculate both measures
entropy_val = calculate_entropy(example_labels)
gini_val = calculate_gini_impurity(example_labels)

print(f"Entropy:      {entropy_val:.4f}")
print(f"Gini Impurity: {gini_val:.4f}")
print()

# Mathematical comparison for the example
p_a, p_b = 0.7, 0.3
print("Manual Calculations:")
print(f"p(A) = {p_a}, p(B) = {p_b}")
print()
print("Entropy = -p(A)×log₂(p(A)) - p(B)×log₂(p(B))")
print(f"Entropy = -{p_a}×{math.log2(p_a):.4f} - {p_b}×{math.log2(p_b):.4f}")
print(f"Entropy = {entropy_val:.4f}")
print()
print("Gini = 1 - p(A)² - p(B)²")
print(f"Gini = 1 - {p_a}² - {p_b}²")
print(f"Gini = 1 - {p_a**2:.4f} - {p_b**2:.4f}")
print(f"Gini = {gini_val:.4f}")

# Compare across different probability distributions
print("\n📊 Comparison Across Different Distributions")
print("=" * 55)

probabilities = np.linspace(0.01, 0.99, 50)
entropy_values = []
gini_values = []

for p in probabilities:
    # Binary case: p and (1-p)
    entropy = -p * math.log2(p) - (1-p) * math.log2(1-p)
    gini = 1 - p**2 - (1-p)**2
    
    entropy_values.append(entropy)
    gini_values.append(gini)

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Entropy vs Gini curves
ax1.plot(probabilities, entropy_values, 'b-', linewidth=2, label='Entropy')
ax1.plot(probabilities, gini_values, 'r-', linewidth=2, label='Gini Impurity')
ax1.set_xlabel('Probability of Class 1')
ax1.set_ylabel('Impurity Measure')
ax1.set_title('Entropy vs Gini Impurity Curves')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Difference between measures
difference = np.array(entropy_values) - np.array(gini_values)
ax2.plot(probabilities, difference, 'g-', linewidth=2)
ax2.set_xlabel('Probability of Class 1')
ax2.set_ylabel('Entropy - Gini')
ax2.set_title('Difference Between Entropy and Gini')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)

# Plot 3: Tennis dataset comparison
features = ['Weather', 'Temperature', 'Humidity', 'Wind']
entropy_gains = []
gini_gains = []

for feature in features:
    # Calculate Information Gain (Entropy-based)
    ig_entropy, _, _ = calculate_information_gain(df, 'Play_Tennis', feature)
    entropy_gains.append(ig_entropy)
    
    # Calculate Gini Gain
    ig_gini, _, _ = calculate_gini_gain(df, 'Play_Tennis', feature)
    gini_gains.append(ig_gini)

x = np.arange(len(features))
width = 0.35

bars1 = ax3.bar(x - width/2, entropy_gains, width, label='Information Gain (Entropy)', color='skyblue')
bars2 = ax3.bar(x + width/2, gini_gains, width, label='Gini Gain', color='lightcoral')

ax3.set_xlabel('Features')
ax3.set_ylabel('Gain')
ax3.set_title('Feature Ranking: Entropy vs Gini')
ax3.set_xticks(x)
ax3.set_xticklabels(features, rotation=45)
ax3.legend()

# Add value labels
for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
    ax3.text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 0.002, 
             f'{entropy_gains[i]:.3f}', ha='center', va='bottom', fontsize=8)
    ax3.text(bar2.get_x() + bar2.get_width()/2, bar2.get_height() + 0.002, 
             f'{gini_gains[i]:.3f}', ha='center', va='bottom', fontsize=8)

# Plot 4: Performance comparison summary
performance_data = {
    'Metric': ['Computation Speed', 'Sensitivity', 'Max Value (Binary)', 'Balanced Trees'],
    'Gini': ['Faster', 'Lower', '0.5', 'Good'],
    'Entropy': ['Slower', 'Higher', '1.0', 'Better']
}

comparison_df = pd.DataFrame(performance_data)
ax4.axis('tight')
ax4.axis('off')

table = ax4.table(cellText=comparison_df.values,
                  colLabels=comparison_df.columns,
                  cellLoc='center',
                  loc='center',
                  colWidths=[0.4, 0.3, 0.3])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Color code the table
for i in range(len(comparison_df.columns)):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax4.set_title('Gini vs Entropy Comparison Table', fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Feature ranking comparison
print("\n🏆 Feature Ranking Comparison")
print("=" * 35)
print(f"{'Feature':<12} | {'Entropy Gain':<12} | {'Gini Gain':<10} | {'Rank Match'}")
print("-" * 55)

entropy_ranks = np.argsort(entropy_gains)[::-1]
gini_ranks = np.argsort(gini_gains)[::-1]

for i, feature in enumerate(features):
    entropy_rank = np.where(entropy_ranks == i)[0][0] + 1
    gini_rank = np.where(gini_ranks == i)[0][0] + 1
    match = "✅" if entropy_rank == gini_rank else "❌"
    
    print(f"{feature:<12} | {entropy_gains[i]:<12.4f} | {gini_gains[i]:<10.4f} | {match}")

# Conclusion
print(f"\n💡 Key Insights:")
print(f"   • Entropy and Gini often produce similar feature rankings")
print(f"   • Gini is computationally more efficient (no logarithms)")
print(f"   • Entropy is more sensitive to probability changes")
print(f"   • Both reach maximum at 50-50 class distribution")
print(f"   • Choice often depends on specific requirements and dataset size")

## 7. Assignment Summary and Conclusions

### 🎯 Key Takeaways

Through this theoretical exploration of Decision Trees, we've covered:

#### **1. Decision Tree Fundamentals**
- Decision Trees are hierarchical, interpretable algorithms
- Best suited for classification and regression problems
- Excellent for problems requiring interpretability
- Handle mixed data types and non-linear relationships

#### **2. Tree Components**
- **Root Node**: Starting point with entire dataset
- **Internal Nodes**: Decision points that split data
- **Leaf Nodes**: Final predictions/classifications
- **Branches**: Connections representing decision outcomes

#### **3. Mathematical Foundations**

**Entropy**: 
- Measures dataset impurity/randomness
- Range: 0 (pure) to log₂(k) (maximum impurity)
- Formula: `Entropy(S) = -Σ pᵢ × log₂(pᵢ)`

**Information Gain**:
- Measures entropy reduction after splitting
- Used for feature selection in tree construction
- Formula: `IG(S,A) = Entropy(S) - Σ(|Sᵥ|/|S|) × Entropy(Sᵥ)`

**Gini Impurity**:
- Alternative impurity measure
- Computationally more efficient than entropy
- Formula: `Gini(S) = 1 - Σ pᵢ²`

#### **4. Practical Applications**
- Feature selection through information gain calculation
- Tree construction using greedy algorithms
- Splitting criteria comparison (Gini vs Entropy)
- Real-world problem solving approach

### 🔬 Experimental Results

From our tennis dataset analysis:
- **Best splitting feature**: Determined by highest information gain
- **Entropy vs Gini**: Similar rankings but different sensitivities
- **Mathematical verification**: Step-by-step calculations confirmed

### 💭 Final Thoughts

Decision Trees provide an excellent balance between **interpretability** and **performance**. The choice between Entropy and Gini Impurity often depends on:

- **Dataset size** (Gini for larger datasets)
- **Computational resources** (Gini is faster)
- **Theoretical requirements** (Entropy for information theory applications)
- **Sensitivity needs** (Entropy is more sensitive to changes)

Understanding these theoretical foundations is crucial for:
- Building effective tree-based models
- Making informed algorithmic choices
- Debugging and optimizing decision trees
- Advancing to ensemble methods (Random Forest, Gradient Boosting)

---

### 📚 Next Steps

1. **Part 2**: Practical implementation with real datasets
2. **Advanced Topics**: Pruning, ensemble methods, feature importance
3. **Optimization**: Hyperparameter tuning and cross-validation
4. **Comparison**: Decision Trees vs other ML algorithms

**Congratulations!** 🎉 You've completed the theoretical foundation of Decision Trees. This knowledge will serve as a solid base for advanced machine learning topics.